In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv
/kaggle/input/feedback-prize-english-language-learning/train.csv
/kaggle/input/feedback-prize-english-language-learning/test.csv


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sn

df=pd.read_csv("../input/feedback-prize-english-language-learning/train.csv")
ss=pd.read_csv("../input/feedback-prize-english-language-learning/sample_submission.csv")
data_test=pd.read_csv("../input/feedback-prize-english-language-learning/test.csv")
data_test= data_test['full_text']
df.columns.values,len(ss.columns.values)

(array(['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary',
        'phraseology', 'grammar', 'conventions'], dtype=object),
 7)

In [12]:
X_train=df['full_text']
y_train=np.array(df.iloc[:,2:])
tokenizer = Tokenizer(num_words=1000, split=' ') 
tokenizer.fit_on_texts(X_train.values)
X_train = tokenizer.texts_to_sequences(X_train.values)
data_test= tokenizer.texts_to_sequences(data_test.values)
X_train = pad_sequences(X_train)
data_test = pad_sequences(data_test)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.1)

In [13]:
model=Sequential()
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 5000, and
# output embedding dimension of size 120.
model.add(Embedding(input_dim=10000, output_dim=120))

# Add a LSTM layer with 128 internal units.
model.add(LSTM(128))
# Add a Dropout layer with persent 80% out units.
model.add(Dropout(0.8))
for i in range(7):
    # Add a Dense layer with 128 units and activation relu.
    model.add(Dense(128,activation='relu'))
    # Add a Dropout layer with persent 70% out units.
    model.add(Dropout(0.8))
# Add a Dense layer with 6 units and activation relu.
model.add(Dense(6,activation='relu'))
#create compile with loss mean_sequard_error and optmizer adm
model.compile(
    loss = 'mse',
    optimizer ='sgd'
)

In [14]:
model.fit(X_train, y_train, epochs = 50)
loss=model.evaluate(X_val,y_val)
loss

Epoch 1/50
110/110 [==============================] - 8s 54ms/step - loss: 9.8419
Epoch 2/50
110/110 [==============================] - 6s 54ms/step - loss: 6.9454
Epoch 3/50
110/110 [==============================] - 6s 53ms/step - loss: 2.6105
Epoch 4/50
110/110 [==============================] - 6s 53ms/step - loss: 1.1161
Epoch 5/50
110/110 [==============================] - 6s 53ms/step - loss: 0.9162
Epoch 6/50
110/110 [==============================] - 6s 53ms/step - loss: 0.8004
Epoch 7/50
110/110 [==============================] - 6s 54ms/step - loss: 0.7491
Epoch 8/50
110/110 [==============================] - 6s 53ms/step - loss: 0.7137
Epoch 9/50
110/110 [==============================] - 6s 53ms/step - loss: 0.6745
Epoch 10/50
110/110 [==============================] - 6s 53ms/step - loss: 0.6500
Epoch 11/50
110/110 [==============================] - 6s 53ms/step - loss: 0.6280
Epoch 12/50
110/110 [==============================] - 6s 53ms/step - loss: 0.6088
Epoch 13/50
1

0.42788341641426086

In [15]:
y_predict=model.predict(data_test)
ss.iloc[:,1:]=y_predict
ss.to_csv('submission.csv',index=False)

In [16]:
ss

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.093692,2.995645,3.200467,3.084649,2.995426,3.043636
1,000BAD50D026,3.093692,2.995645,3.200467,3.084649,2.995426,3.043636
2,00367BB2546B,3.093692,2.995645,3.200467,3.084649,2.995426,3.043636
